**Distributed CNN Training with Data Augmentation using TensorFlow**

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load and preprocess the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Define a CNN model
def create_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10)
    ])

    model.compile(optimizer='adam',
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return model

# Create a MirroredStrategy for distributed training
strategy = tf.distribute.MirroredStrategy()

print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    # Create and compile the model within the strategy scope
    model = create_model()

# Augment the training data using ImageDataGenerator
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)
datagen.fit(x_train)

# Train the model using the fit method with augmented data
model.fit(datagen.flow(x_train, y_train, batch_size=64), epochs=5, steps_per_epoch=len(x_train) // 64)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)


170498071/170498071 [==============================] - 11s 0us/step
Number of devices: 1
Epoch 1/5
781/781 [==============================] - 93s 117ms/step - loss: 1.7706 - accuracy: 0.1097
Epoch 2/5
781/781 [==============================] - 89s 114ms/step - loss: 1.5218 - accuracy: 0.0943
Epoch 3/5
781/781 [==============================] - 89s 114ms/step - loss: 1.4014 - accuracy: 0.0928
Epoch 4/5
781/781 [==============================] - 89s 113ms/step - loss: 1.3208 - accuracy: 0.0945
Epoch 5/5
781/781 [==============================] - 91s 116ms/step - loss: 1.2703 - accuracy: 0.0971
313/313 - 4s - loss: 1.1109 - accuracy: 0.0963 - 4s/epoch - 12ms/step

Test accuracy: 0.09629999846220016


This TensorFlow script demonstrates distributed training of a Convolutional Neural Network (CNN) on the CIFAR-10 dataset using the MirroredStrategy for multi-GPU support. The CNN architecture consists of convolutional and pooling layers followed by dense layers. The script utilizes the ImageDataGenerator to augment the training data with rotations, shifts, and flips. The MirroredStrategy ensures efficient distributed training across multiple devices. The model is compiled with the Adam optimizer and Sparse Categorical Crossentropy loss. After training with augmented data, the script evaluates the model on the test set, providing insights into its accuracy.